In [295]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

from collections import defaultdict
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

from WA_state_API_functions import get_committee_member_data

# Load

In [265]:
engine = create_engine('postgresql://localhost:5432/wa_leg_raw')

In [266]:
vote_df = pd.read_sql_query('select * from "vote_api"',con=engine)

In [267]:
committee_member_df = pd.read_sql_query('select * from "committee_member_api"',con=engine)

# Start legislators_df

In [268]:
legislators_df = vote_df.loc[:, ['biennium', 'voter_id', 'voter_name', 'voting_agency']]

In [269]:
legislators_df.drop_duplicates(keep='first', inplace=True)

In [270]:
legislators_df.head()

,biennium,voter_id,voter_name,voting_agency
0,1991-92,7,Anderson,House
1,1991-92,11,Appelwick,House
2,1991-92,17,Ballard,House
3,1991-92,23,Basich,House
4,1991-92,32,Beck,House


In [271]:
cm_legislators_df = committee_member_df.loc[:, ['district', 'id', 'name', 'first_name', 'party']]

cm_legislators_df = cm_legislators_df.rename(index=str, columns={"id": "voter_id"})

cm_legislators_df['last_name'] = cm_legislators_df['name'].apply(lambda x: x.split()[1])

cm_legislators_df.drop_duplicates(keep='first', inplace=True)

legislators_df = legislators_df.merge(cm_legislators_df, how='outer', on='voter_id')

legislators_df.drop_duplicates(keep="first", inplace=True)

In [272]:
legislators_df.head()

,biennium,voter_id,voter_name,voting_agency,district,name,first_name,party,last_name
0,1991-92,7,Anderson,House,NaN,NaN,NaN,NaN,NaN
1,1993-94,7,Anderson,House,NaN,NaN,NaN,NaN,NaN
2,1995-96,7,"Anderson, C.",Senate,NaN,NaN,NaN,NaN,NaN
3,1991-92,11,Appelwick,House,46,Marlin Appelwick,Marlin,Democrat,Appelwick
4,1993-94,11,Appelwick,House,46,Marlin Appelwick,Marlin,Democrat,Appelwick


# Create missing_legislators_manually

In [273]:
manual = [{'name': 'Ann Anderson', 'voter_id': '7', 'district': '42', 'party': 'R'},
 {'name': 'Bob Basich', 'voter_id': '23', 'district': '19', 'party': 'D'},
 {'name': 'John Beck', 'voter_id': '32', 'district': '21', 'party': 'R'},
 {'name': 'Jennifer Belcher', 'voter_id': '34', 'district': '22', 'party': 'D'},
 {'name': 'John Betrozoff', 'voter_id': '40', 'district': '45', 'party': 'R'},
 {'name': 'Rose Bowman', 'voter_id': '45', 'district': '20', 'party': 'R'},
 {'name': 'Dennis Braddock', 'voter_id': '46',  'district': '42', 'party': 'D'},
 {'name': 'Lane Bray', 'voter_id': '47', 'district': '8', 'party': 'D'},
 {'name': 'Joanne Brekke', 'voter_id': '48', 'district': '32', 'party': 'D'},
 {'name': 'Art Broback', 'voter_id': '50', 'district': '28', 'party': 'R'},
 {'name': 'Jean-Marie Brough', 'voter_id': '54', 'district': '30', 'party': 'R'},
 {'name': 'Bill Brumsickle', 'voter_id': '63', 'district': '20', 'party': 'R'},
 {'name': 'Maria Cantwell', 'voter_id': '68', 'district': '44', 'party': 'D'},
 {'name': 'Gary Chandler', 'voter_id': '75', 'district': '13', 'party': 'R'},
 {'name': 'David Cooper', 'voter_id': '93', 'district': '18', 'party': 'D'},
 {'name': 'Bill Day', 'voter_id': '103', 'district': '3', 'party': 'D'},
 {'name': 'Dennis Dellwo', 'voter_id': '110', 'district': '3', 'party': 'D'},
 {'name': 'Randy Dorn', 'voter_id': '116', 'district': '2', 'party': 'D'},
 {'name': 'Brian Ebersole', 'voter_id': '121', 'district': '29', 'party': 'D'}, 
 {'name': 'Betty Edmondson', 'voter_id': '123', 'district': '14', 'party': 'R'},
 {'name': 'Roy Ferguson', 'voter_id': '132', 'district': '48', 'party': 'R'},
 {'name': 'Greg Fisher', 'voter_id': '135', 'district': '33', 'party': 'D'},
 {'name': 'Elmira Forner', 'voter_id': '140', 'district': '47', 'party': 'R'}, 
 {'name': 'Steve Fuhrman', 'voter_id': '146', 'district': '7', 'party': 'R'},
 {'name': 'Lorraine Hine', 'voter_id': '186', 'district': '33', 'party': 'D'},
 {'name': 'Bruce Holland', 'voter_id': '192', 'district': '47', 'party': 'R'},
 {'name': 'Jay Inslee', 'voter_id': '205', 'district': '14', 'party': 'D'},
 {'name': 'Rob Johnson', 'voter_id': '214', 'district': '40', 'party': 'D'},
 {'name': 'Evan Jones', 'voter_id': '219', 'district': '24', 'party': 'D'},
 {'name': 'Joseph King', 'voter_id': '226', 'district': '49', 'party': 'D'},
 {'name': 'Richard King', 'voter_id': '227', 'district': '38', 'party': 'D'},
 {'name': 'Pete Kremen', 'voter_id': '234', 'district': '42', 'party': 'D'},
 {'name': 'June Leonard', 'voter_id': '245', 'district': '11', 'party': 'D'},
 {'name': 'Gary Locke', 'voter_id': '251', 'district': '37', 'party': 'D'},
 {'name': 'Curtis Ludwig', 'voter_id': '254', 'district': '8', 'party': 'D'},
 {'name': 'Fred May', 'voter_id': '264', 'district': '41', 'party': 'R'},
 {'name': 'Alex McLean', 'voter_id': '276', 'district': '12', 'party': 'R'},
 {'name': 'Holly Meyers', 'voter_id': '282', 'district': '17', 'party': 'D'},
 {'name': 'Todd Mielke', 'voter_id': '284', 'district': '5', 'party': 'R'},
 {'name': 'Todd Mielke', 'voter_id': '284', 'district': '6', 'party': 'R'},
 {'name': 'Louise Miller', 'voter_id': '285', 'district': '45', 'party': 'R'},
 {'name': 'John Moyer', 'voter_id': '297', 'district': '6', 'party': 'R'},
 {'name': 'John Moyer', 'voter_id': '297', 'district': '3', 'party': 'R'},
 {'name': 'Harvey Myers', 'voter_id': '301', 'district': '15', 'party': 'R'},
 {'name': 'Darwin Nealey', 'voter_id': '304', 'district': '9', 'party': 'R'},
 {'name': "John O'Brien", 'voter_id': '311', 'district': '37', 'party': 'D'},
 {'name': 'George Orr', 'voter_id': '320', 'district': '4', 'party': 'D'},
 {'name': 'George-Marshall Paris', 'voter_id': '325', 'district': '44', 'party': 'R'},
 {'name': 'W-Kim Peery', 'voter_id': '328', 'district': '17', 'party': 'D'},
 {'name': 'Larry Phillips', 'voter_id': '332', 'district': '36', 'party': 'D'},
 {'name': 'Wes Pruitt', 'voter_id': '341', 'district': '26', 'party': 'D'},
 {'name': 'Margaret Rayburn', 'voter_id': '349', 'district': '15', 'party': 'D'},
 {'name': 'Mike Riley', 'voter_id': '359', 'district': '19', 'party': 'D'},
 {'name': 'Judith Roland', 'voter_id': '363', 'district': '31', 'party': 'D'},
 {'name': 'Nancy Rust', 'voter_id': '367', 'district': '1', 'party': 'D'},
 {'name': 'Karen Schmidt', 'voter_id': '376', 'district': '23', 'party': 'R'},
 {'name': 'B-Jean Silver', 'voter_id': '396', 'district': '5', 'party': 'R'},
 {'name': 'B-Jean Silver', 'voter_id': '396', 'district': '6', 'party': 'R'},
 {'name': 'Duane Sommers, ', 'voter_id': '406', 'district': '6', 'party': 'R'},
 {'name': 'Art Sprenkle', 'voter_id': '410', 'district': '39', 'party': 'D'},
 {'name': 'Randy Tate', 'voter_id': '429', 'district': '25', 'party': 'R'},
 {'name': 'Christopher Vance', 'voter_id': '444', 'district': '31', 'party': 'R'}, 
 {'name': 'Art Wang', 'voter_id': '449', 'district': '27', 'party': 'D'},
 {'name': 'Simeon Wilson', 'voter_id': '472', 'district': '10', 'party': 'R'},
 {'name': 'Jesse Wineberry', 'voter_id': '474', 'district': '43', 'party': 'D'},
 {'name': 'Jesse Wineberry', 'voter_id': '474', 'district': '37', 'party': 'D'},
 {'name': 'John Wynne', 'voter_id': '481', 'district': '39', 'party': 'R'},
 {'name': 'Neil Amondson', 'voter_id': '5', 'district': '20', 'party': 'R'},
 {'name': 'Cliff Bailey', 'voter_id': '15', 'district': '39', 'party': 'R'},
 {'name': 'Scott Barr', 'voter_id': '19', 'district': '7', 'party': 'R'},
 {'name': 'Albert Bauer', 'voter_id': '25', 'district': '49', 'party': 'D'},
 {'name': 'Alan Bluechel', 'voter_id': '42', 'district': '45', 'party': 'R'},
 {'name': 'Emilio Cantu', 'voter_id': '67', 'district': '41', 'party': 'R'},
 {'name': 'Paul Conner', 'voter_id': '90', 'district': '24', 'party': 'D'},
 {'name': 'Ellen Craswell', 'voter_id': '95', 'district': '23', 'party': 'R'},
 {'name': 'Time Erwin', 'voter_id': '127', 'district': '44', 'party': 'R'},
 {'name': 'Marcus Gaspard', 'voter_id': '148', 'district': '25', 'party': 'D'},
 {'name': 'Frank Hansen', 'voter_id': '168', 'district': '13', 'party': 'D'},
 {'name': 'Jeannette Hayner', 'voter_id': '176', 'district': '16', 'party': 'R'},
 {'name': 'Jim Jesernig', 'voter_id': '211', 'district': '8', 'party': 'D'},
 {'name': 'Lela Kreidler,', 'voter_id': '232', 'district': '22', 'party': 'D'},
 {'name': 'Ken Madsen', 'voter_id': '256', 'district': '2', 'party': 'D'},
 {'name': 'Jim Matson', 'voter_id': '261', 'district': '14', 'party': 'R'},
 {'name': 'Patrick McMullen', 'voter_id': '278', 'district': '40', 'party': 'D'},
 {'name': 'Jack Metcalf', 'voter_id': '281', 'district': '10', 'party': 'R'},
 {'name': 'Ray Moore', 'voter_id': '290', 'district': '36', 'party': 'D'},
 {'name': 'Patty Murray', 'voter_id': '299', 'district': '1', 'party': 'D'},
 {'name': 'Gary Nelson', 'voter_id': '308', 'district': '21', 'party': 'R'},
 {'name': 'Janice Niemi', 'voter_id': '310', 'district': '43', 'party': 'D'},
 {'name': 'E-G Patterson', 'voter_id': '326', 'district': '9', 'party': 'R'},
 {'name': 'Dwight Pelz', 'voter_id': '329', 'district': '37', 'party': 'D'},
 {'name': 'A-L Rasmussen', 'voter_id': '345', 'district': '29', 'party': 'D'},
 {'name': 'Nina Rinehart', 'voter_id': '360', 'district': '46', 'party': 'D'},
 {'name': 'Gerald Saling', 'voter_id': '370', 'district': '5', 'party': 'R'},
 {'name': 'George Sellar', 'voter_id': '387', 'district': '12', 'party': 'R'},
 {'name': 'Sylvia Skratek', 'voter_id': '399', 'district': '47', 'party': 'D'},
 {'name': 'Adam Smith', 'voter_id': '402', 'district': '33', 'party': 'D'},
 {'name': 'Linda Smith', 'voter_id': '403', 'district': '18', 'party': 'R'},
 {'name': 'Lois Stratton', 'voter_id': '414', 'district': '3', 'party': 'D'},
 {'name': 'Dean Sutherland', 'voter_id': '423', 'district': '17', 'party': 'D'},
 {'name': 'Phil Talmadge', 'voter_id': '428', 'district': '34', 'party': 'D'},
 {'name': 'Leo Thorsness', 'voter_id': '434', 'district': '11', 'party': 'R'},
 {'name': 'Larry Vognild', 'voter_id': '445', 'district': '38', 'party': 'D'},
 {'name': 'Peter von-Reichbauer', 'voter_id': '446', 'district': '30', 'party': 'R'},
 {'name': 'Alvin Williams', 'voter_id': '467', 'district': '32', 'party': 'D'},
 {'name': 'R-Lorraine Wojahn', 'voter_id': '477', 'district': '27', 'party': 'D'},
 {'name': 'Myron Kreidler', 'voter_id': '233', 'district': '22', 'party': 'D'},
 {'name': 'Wanda Hansen', 'voter_id': '169', 'district': '13', 'party': 'D'},
 {'name': 'Susan Sumner', 'voter_id': '419', 'district': '28', 'party': 'R'},
 {'name': 'Barbara Holm', 'voter_id': '639', 'district': '35', 'party': 'D'},
 {'name': 'Kathleen Drew', 'voter_id': '570', 'district': '5', 'party': 'D'},
 {'name': 'Valoria Loveland', 'voter_id': '562', 'district': '16', 'party': 'D'},
 {'name': 'Kevin Quigley', 'voter_id': '563', 'district': '39', 'party': 'D'},
 {'name': 'David Chappell', 'voter_id': '621', 'district': '20', 'party': 'D'},
 {'name': 'Barbara Cothern', 'voter_id': '629', 'district': '1', 'party': 'D'},
 {'name': 'Stanley Flemming', 'voter_id': '636', 'district': '28', 'party': 'D'},
 {'name': 'Dale Foreman', 'voter_id': '637', 'district': '12', 'party': 'R'},
 {'name': 'Mick Hansen', 'voter_id': '638', 'district': '13', 'party': 'D'},
 {'name': 'Jim Johanson', 'voter_id': '640', 'district': '44', 'party': 'D'},
 {'name': 'Linda Johnson', 'voter_id': '641', 'district': '1', 'party': 'D'},
 {'name': 'Sue Karahalios', 'voter_id': '642', 'district': '10', 'party': 'D'},
 {'name': 'Dave Lemmon', 'voter_id': '645', 'district': '14', 'party': 'D'},
 {'name': 'Jim Springer', 'voter_id': '656', 'district': '18', 'party': 'D'},
 {'name': 'Brian Thomas', 'voter_id': '660', 'district': '5', 'party': 'R'},
 {'name': 'Cathy Wolfe', 'voter_id': '662', 'district': '22', 'party': 'D'},
 {'name': 'Bill Backlund', 'voter_id': '1369', 'district': '45', 'party': 'R'},
 {'name': 'Vivian Caver', 'voter_id': '1408', 'district': '37', 'party': 'D'},
 {'name': 'Thomas Moak', 'voter_id': '1351', 'district': '8', 'party': 'D'},
 {'name': 'Ray Schow', 'voter_id': '1352', 'district': '30', 'party': 'R'},
 {'name': 'Barney Beeksma', 'voter_id': '1582', 'district': '10', 'party': 'R'},
 {'name': 'Jerry Blanton', 'voter_id': '1569', 'district': '21', 'party': 'R'},
 {'name': 'Gene Goldsmith', 'voter_id': '1575', 'district': '42', 'party': 'R'},
 {'name': 'Steve Hargrove', 'voter_id': '1588', 'district': '23', 'party': 'R'},
 {'name': 'Tomm Huff', 'voter_id': '1578', 'district': '26', 'party': 'R'},
 {'name': 'Kathy Lambert', 'voter_id': '1589', 'district': '45', 'party': 'R'},
 {'name': 'John Pennington', 'voter_id': '1591', 'district': '18', 'party': 'R'},
 {'name': 'Renee Radcliff', 'voter_id': '1579', 'district': '21', 'party': 'R'},
 {'name': 'Mike Sherstad', 'voter_id': '1593', 'district': '1', 'party': 'R'},
 {'name': 'Darlene Fairley', 'voter_id': '1603', 'district': '32', 'party': 'D'},
 {'name': 'Hal Palmer', 'voter_id': '1667', 'district': '18', 'party': 'R'},
 {'name': 'Calvin Goings', 'voter_id': '1938', 'district': '25', 'party': 'D'},
 {'name': 'Carl Scheuerman', 'voter_id': '1949', 'district': '29', 'party': 'D'},
 {'name': 'Brian Sullivan', 'voter_id': '2132', 'district': '29', 'party': 'D'},
 {'name': 'Mike Wensman', 'voter_id': '2222', 'district': '41', 'party': 'R'},
 {'name': 'Lena Swanson', 'voter_id': '2202', 'district': '35', 'party': 'D'},
 {'name': 'Glenn Anderson', 'voter_id': '2147', 'district': '5', 'party': 'R'},
 {'name': 'John Blalock', 'voter_id': '2296', 'district': '33', 'party': 'D'},
 {'name': 'Carolyn Edmonds', 'voter_id': '3471', 'district': '32', 'party': 'D'},
 {'name': 'Michael Stensen', 'voter_id': '3485', 'district': '31', 'party': 'D'},
 {'name': 'Joe Marine', 'voter_id': '6307', 'district': '21', 'party': 'R'},
 {'name': 'John Wyss', 'voter_id': '11176', 'district': '6', 'party': 'R'},
 {'name': 'Eric Oemig', 'voter_id': '12067', 'district': '45', 'party': 'D'},
 {'name': 'Gina McCabe', 'voter_id': '20759', 'district': '14', 'party': 'R'},
 {'name': 'William Jenkin', 'voter_id': '26172', 'district': '16', 'party': 'R'},
 {'name': 'Keith Wagoner', 'voter_id': '28317', 'district': '39', 'party': 'R'},
 {'name': 'Stanley Johnson', 'voter_id': '420', 'district': '28', 'party':'R'},
 {'name': 'Ron Carlson', 'voter_id': '877', 'district': '47', 'party':'R'},
 {'name': 'Georgette Valle', 'voter_id': '442', 'district': '34', 'party':'D'},
 {'name': 'Richard Neher', 'voter_id': '305', 'district': '16', 'party':'R'}, 
 {'name': 'Richard Nelson', 'voter_id': '307', 'district': '32', 'party':'D'}]

In [274]:
manual_df = pd.DataFrame(manual)

In [275]:
manual_df['first_name'] = manual_df['name'].apply(lambda x: x.split()[0])

In [276]:
manual_df['last_name'] = manual_df['name'].apply(lambda x: x.split()[1])

In [277]:
manual_df = manual_df.drop('name', axis=1)

In [278]:
manual_df.to_csv('missing_legislators.csv', sep='|', index=False)

# Load missing_legislator_info from csv

In [279]:
missing_legislator_info_df = pd.read_csv('missing_legislators.csv', sep='|')

In [280]:
missing_legislator_info_df.head()

,district,party,voter_id,first_name,last_name
0,42,R,7,Ann,Anderson
1,19,D,23,Bob,Basich
2,21,R,32,John,Beck
3,22,D,34,Jennifer,Belcher
4,45,R,40,John,Betrozoff


In [281]:
missing_legislator_info_df['district'] = missing_legislator_info_df['district'].apply(str)

In [282]:
missing_legislator_info_df['voter_id'] = missing_legislator_info_df['voter_id'].apply(str)

# Merge legislators_df to missing_legislators

In [286]:
legislators_df = legislators_df.merge(missing_legislator_info_df, how='outer', on='voter_id', suffixes=('', '_missing'))

In [287]:
legislators_df.head()

,biennium,voter_id,voter_name,voting_agency,district,name,first_name,party,last_name,district_missing,party_missing,first_name_missing,last_name_missing
0,1991-92,7,Anderson,House,NaN,NaN,NaN,NaN,NaN,42.0,R,Ann,Anderson
1,1993-94,7,Anderson,House,NaN,NaN,NaN,NaN,NaN,42.0,R,Ann,Anderson
2,1995-96,7,"Anderson, C.",Senate,NaN,NaN,NaN,NaN,NaN,42.0,R,Ann,Anderson
3,1991-92,11,Appelwick,House,46.0,Marlin Appelwick,Marlin,Democrat,Appelwick,NaN,NaN,NaN,NaN
4,1993-94,11,Appelwick,House,46.0,Marlin Appelwick,Marlin,Democrat,Appelwick,NaN,NaN,NaN,NaN


In [233]:
def replace_missing_first_name(x):
    if type(x['first_name']) == float:
        return x['first_name_missing']
    else:
        return x['first_name']

In [234]:
def replace_missing_party(x):
    if type(x['party']) == float:
        if x['party_missing'] == 'R':
            return 'Republican'
        if x['party_missing'] == 'D':
            return 'Democrat'
    else:
        return x['party']

In [235]:
def replace_missing_district(x):
    if type(x['district']) == float:
        return x['district_missing']
    else:
        return x['district']

In [236]:
def replace_missing_last_name(x):
    if type(x['last_name']) == float:
        return x['last_name_missing']
    else:
        return x['last_name']

In [237]:
legislators_df['first_name'] = legislators_df.apply(lambda x : replace_missing_first_name(x),axis=1)
legislators_df['party'] = legislators_df.apply(lambda x : replace_missing_party(x),axis=1)
legislators_df['district'] = legislators_df.apply(lambda x : replace_missing_district(x),axis=1)
legislators_df['last_name'] = legislators_df.apply(lambda x : replace_missing_last_name(x),axis=1)

In [241]:
legislators_df

,biennium,voter_id,voter_name,voting_agency,district,name,first_name,party,last_name,district_missing,party_missing,first_name_missing,last_name_missing
0,1991-92,7,Anderson,House,42,NaN,Ann,Republican,Anderson,42,R,Ann,Anderson
1,1993-94,7,Anderson,House,42,NaN,Ann,Republican,Anderson,42,R,Ann,Anderson
2,1995-96,7,"Anderson, C.",Senate,42,NaN,Ann,Republican,Anderson,42,R,Ann,Anderson
3,1991-92,11,Appelwick,House,46,Marlin Appelwick,Marlin,Democrat,Appelwick,NaN,NaN,NaN,NaN
4,1993-94,11,Appelwick,House,46,Marlin Appelwick,Marlin,Democrat,Appelwick,NaN,NaN,NaN,NaN
5,1995-96,11,Appelwick,House,46,Marlin Appelwick,Marlin,Democrat,Appelwick,NaN,NaN,NaN,NaN
6,1997-98,11,Appelwick,House,46,Marlin Appelwick,Marlin,Democrat,Appelwick,NaN,NaN,NaN,NaN
7,1991-92,17,Ballard,House,12,Clyde Ballard,Clyde,Republican,Ballard,NaN,NaN,NaN,NaN
8,1993-94,17,Ballard,House,12,Clyde Ballard,Clyde,Republican,Ballard,NaN,NaN,NaN,NaN
9,1995-96,17,Ballard,House,12,Clyde Ballard,Clyde,Republican,Ballard,NaN,NaN,NaN,NaN


In [147]:
legislators_df[legislators_df['first_name'].isnull()]

,biennium,voter_id,voter_name,voting_agency,district,name,first_name,party,last_name,district_missing,name_missing,party_missing,first_name_missing


In [245]:
legislators_df = legislators_df.drop(['voter_name', 'name', 'district_missing', 'party_missing', 'first_name_missing', 'last_name_missing', 'biennium'], axis=1)

In [246]:
legislators_df.drop_duplicates(keep='first', inplace=True)

In [249]:
legislators_df.columns = ['id', 'agency', 'district', 'first_name', 'party', 'last_name']

In [250]:
legislators_df.head()

,id,agency,district,first_name,party,last_name
0,7,House,42,Ann,Republican,Anderson
2,7,Senate,42,Ann,Republican,Anderson
3,11,House,46,Marlin,Democrat,Appelwick
7,17,House,12,Clyde,Republican,Ballard
13,23,House,19,Bob,Democrat,Basich


In [254]:
len(legislators_df['id'].unique())

531

In [255]:
len(legislators_df)

647

In [296]:
def create_legislator_df():
    """Merge votes_df, committee_df and missing_legislator_info_df and clean data to create legislator_df
    
    Input
    votes_df: pandas dataframe loaded from wa_leg_raw database, vote_api table
    committee_member_df: pandas dataframe loaded from wa_leg_raw database, committee_member_api table
    missing_legislator_info_df: pandas dataframe loaded missing_legislators.csv
    """

    engine = create_engine('postgresql://localhost:5432/wa_leg_raw')
    vote_df = pd.read_sql_query('select * from "vote_api"',con=engine)
    committee_member_df = pd.read_sql_query('select * from "committee_member_api"',con=engine)
    
    leg_info_from_vote_df = vote_df.loc[:, ['biennium', 'voter_id', 'voter_name', 'voting_agency']]
    leg_info_from_vote_df.drop_duplicates(keep='first', inplace=True)
    
    leg_info_from_cm_df = committee_member_df.loc[:, ['district', 'id', 'name', 'first_name', 'party']]
    leg_info_from_cm_df = leg_info_from_cm_df.rename(index=str, columns={"id": "voter_id"})
    leg_info_from_cm_df['last_name'] = leg_info_from_cm_df['name'].apply(lambda x: x.split()[1])
    leg_info_from_cm_df.drop_duplicates(keep='first', inplace=True)

    legislator_df = leg_info_from_vote_df.merge(leg_info_from_cm_df, how='outer', on='voter_id')
    legislator_df.drop_duplicates(keep="first", inplace=True)
    
    missing_leg_info_df = pd.read_csv('missing_legislators.csv', sep='|')
    missing_leg_info_df['district'] = missing_leg_info_df['district'].apply(str)
    missing_leg_info_df['voter_id'] = missing_leg_info_df['voter_id'].apply(str)
    
    legislator_df = legislator_df.merge(missing_leg_info_df, how='outer', on='voter_id', suffixes=('', '_missing'))
    
    def replace_missing_first_name(x):
        if type(x['first_name']) == float:
            return x['first_name_missing']
        else:
            return x['first_name']
        
    def replace_missing_last_name(x):
        if type(x['last_name']) == float:
            return x['last_name_missing']
        else:
            return x['last_name']
        
    def replace_missing_party(x):
        if type(x['party']) == float:
            if x['party_missing'] == 'R':
                return 'Republican'
            if x['party_missing'] == 'D':
                return 'Democrat'
        else:
            return x['party']
        
    def replace_missing_district(x):
        if type(x['district']) == float:
            return x['district_missing']
        else:
            return x['district']
        
    legislator_df['first_name'] = legislator_df.apply(lambda x : replace_missing_first_name(x),axis=1)
    legislator_df['party']      = legislator_df.apply(lambda x : replace_missing_party(x),     axis=1)
    legislator_df['district']   = legislator_df.apply(lambda x : replace_missing_district(x),  axis=1)
    legislator_df['last_name']  = legislator_df.apply(lambda x : replace_missing_last_name(x), axis=1)
    
    legislator_df = legislator_df.drop(['voter_name', 'name', 'district_missing', 'party_missing', 'first_name_missing', 'last_name_missing', 'biennium'], axis=1)
    legislator_df.drop_duplicates(keep='first', inplace=True)
    legislator_df.columns = ['id', 'agency', 'district', 'first_name', 'party', 'last_name']
    legislator_df['id'] = legislator_df['id'].apply(int)
    legislator_df['district'] = legislator_df['district'].apply(int)
    
    return legislator_df

In [289]:
leg_df = create_legislator_df()

In [291]:
leg_df.head()

,id,agency,district,first_name,party,last_name
0,7,House,42,Ann,Republican,Anderson
2,7,Senate,42,Ann,Republican,Anderson
3,11,House,46,Marlin,Democrat,Appelwick
7,17,House,12,Clyde,Republican,Ballard
13,23,House,19,Bob,Democrat,Basich


In [314]:
class CreateLegislatorDataframe(BaseEstimator, TransformerMixin):
    """Merge votes_df, committee_df and missing_legislator_df and clean data to create legislator_df"""
    
    def __init__(self, vote_df, committee_member_df, missing_leg_info_df):
        """Input
           votes_df: pandas dataframe loaded from wa_leg_raw database, vote_api table
           committee_member_df: pandas dataframe loaded from wa_leg_raw database, committee_member_api table
           missing_legislator_df: pandas dataframe loaded missing_legislators.csv"""
        self.vote_df = vote_df
        self.committee_member_df = committee_member_df
        self.missing_leg_info_df = missing_leg_info_df
    
    def fit(self, X=None, y=None):
        return self

    def transform(self, X=None):
        leg_info_from_vote_df = self.vote_df.loc[:, ['biennium', 'voter_id', 'voter_name', 'voting_agency']]
        leg_info_from_vote_df.drop_duplicates(keep='first', inplace=True)

        leg_info_from_cm_df = self.committee_member_df.loc[:, ['district', 'id', 'name', 'first_name', 'party']]
        leg_info_from_cm_df = leg_info_from_cm_df.rename(index=str, columns={"id": "voter_id"})
        leg_info_from_cm_df['last_name'] = leg_info_from_cm_df['name'].apply(lambda x: x.split()[1])
        leg_info_from_cm_df.drop_duplicates(keep='first', inplace=True)

        legislator_df = leg_info_from_vote_df.merge(leg_info_from_cm_df, how='outer', on='voter_id')
        legislator_df.drop_duplicates(keep="first", inplace=True)

        self.missing_leg_info_df['district'] = self.missing_leg_info_df['district'].apply(str)
        self.missing_leg_info_df['voter_id'] = self.missing_leg_info_df['voter_id'].apply(str)

        legislator_df = legislator_df.merge(self.missing_leg_info_df, how='outer', on='voter_id', suffixes=('', '_missing'))

        legislator_df['first_name'] = legislator_df.apply(lambda x : replace_missing_first_name(x),axis=1)
        legislator_df['party']      = legislator_df.apply(lambda x : replace_missing_party(x),     axis=1)
        legislator_df['district']   = legislator_df.apply(lambda x : replace_missing_district(x),  axis=1)
        legislator_df['last_name']  = legislator_df.apply(lambda x : replace_missing_last_name(x), axis=1)

        legislator_df = legislator_df.drop(['voter_name', 'name', 'district_missing', 'party_missing', 'first_name_missing', 'last_name_missing', 'biennium'], axis=1)
        legislator_df.drop_duplicates(keep='first', inplace=True)
        legislator_df.columns = ['id', 'agency', 'district', 'first_name', 'party', 'last_name']
        legislator_df['id'] = legislator_df['id'].apply(int)
        legislator_df['district'] = legislator_df['district'].apply(int)

        return legislator_df
    
    
    @staticmethod
    def replace_missing_first_name(x):
        if type(x['first_name']) == float:
            return x['first_name_missing']
        else:
            return x['first_name']
        
        
    @staticmethod
    def replace_missing_last_name(x):
        if type(x['last_name']) == float:
            return x['last_name_missing']
        else:
            return x['last_name']
        
        
    @staticmethod
    def replace_missing_party(x):
        if type(x['party']) == float:
            if x['party_missing'] == 'R':
                return 'Republican'
            if x['party_missing'] == 'D':
                return 'Democrat'
        else:
            return x['party']
        
        
    @staticmethod
    def replace_missing_district(x):
        if type(x['district']) == float:
            return x['district_missing']
        else:
            return x['district']

    

In [315]:
create = CreateLegislatorDataframe(vote_df, committee_member_df, missing_legislator_info_df)

In [316]:
create.fit()

CreateLegislatorDataframe(committee_member_df=     acronym  agency biennium             committee_name district first_name  \
0       ANDE  Senate  1993-94      Government Operations       42        Ann
1       PELE   House  1995-96             Appropriations       25      Grant
2       ELLI   House  1995-96           ...526  Republican
2527    Democrat
2528  Republican
2529  Republican

[2530 rows x 11 columns],
             missing_leg_info_df=     district party  voter_id  first_name   last_name
0          42     R         7         Ann    Anderson
1          19     D        23         Bob      Basich
2          21     R        32        John        Beck
3          22     D        34    Jennifer     Belcher
4         ...   Richard       Neher
159        32     D       307     Richard      Nelson

[160 rows x 5 columns],
             vote_df=        biennium   bill_id         motion sequence_number     vote  \
0        1991-92  SHB 1001  Final Passage               4      Yea
1      

In [317]:
leg_test = create.transform()

In [318]:
leg_test

,id,agency,district,first_name,party,last_name
0,7,House,42,Ann,Republican,Anderson
2,7,Senate,42,Ann,Republican,Anderson
3,11,House,46,Marlin,Democrat,Appelwick
7,17,House,12,Clyde,Republican,Ballard
13,23,House,19,Bob,Democrat,Basich
16,32,House,21,John,Republican,Beck
17,34,House,22,Jennifer,Democrat,Belcher
18,40,House,45,John,Republican,Betrozoff
19,45,House,20,Rose,Republican,Bowman
20,46,House,42,Dennis,Democrat,Braddock
